# PG Vector

This notebook shows how to use [pgvector](https://www.pinecone.io/) extension for Postgres to index and search for similar papers based on their embeddings.

**Strengths:**
- Built as an extension for PostgresSQL, leveraging its relational capabilities.
- Utilizes SQL for querying, making it familiar to database users.

**Weaknesses:**
- Performance might not be as high as purpose-built vector databases for certain use cases.
- Vectors are not natively supported by Postgres. An extension is needed to store and query vectors.
- Requires more manual optimization and tuning for specific workloads.

In [3]:
import os

os.getcwd()

'/home/jupyter-patrickzierahn'

In [4]:
import pandas as pd
import numpy as np

datafile_path = f"data/software_architecture/bib-text_embeddings.csv.csv"
df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(eval).apply(np.array)  # convert string to array
df

,author,title,doi,classes,url,abstract,embedding
0,Alessio Bucaioni and Patrizio Pelliccione and ...,Aligning Architecture with Business Goals in t...,10.1109/ICSA51549.2021.00020,"Meta Data{Research Level{Primary Research}, Ki...",https://doi.org/10.1109/ICSA51549.2021.00020,When designing complex automotive systems in p...,"[0.00016247970052063465, -0.006258538924157619..."
1,H{\'{e}}ctor Cadavid and Vasilios Andrikopoulo...,System- and Software-level Architecting Harmon...,10.1109/ICSA51549.2021.00010,"Meta Data{Kind{full}, Paper class{Evaluation R...",https://doi.org/10.1109/ICSA51549.2021.00010,The problems caused by the gap between system-...,"[0.01373602356761694, -0.007272415794432163, 0..."
2,Joshua Garcia and Mehdi Mirakhorli and Lu Xiao...,Constructing a Shared Infrastructure for Softw...,10.1109/ICSA51549.2021.00022,"Meta Data{Paper class{Evaluation Research}, Re...",https://doi.org/10.1109/ICSA51549.2021.00022,Over the past three decades software engineeri...,"[0.0022478506434708834, -0.0009823687141761184..."
3,Holger Knoche and Wilhelm Hasselbring,Continuous {API} Evolution in Heterogenous Ent...,10.1109/ICSA51549.2021.00014,"Meta Data{Research Level{Primary Research}, Ki...",https://doi.org/10.1109/ICSA51549.2021.00014,The ability to independently deploy parts of a...,"[0.0058586616069078445, -0.031336553394794464,..."
4,Duc Minh Le and Suhrid Karthik and Marcelo Sch...,Architectural Decay as Predictor of Issue- and...,10.1109/ICSA51549.2021.00017,"Meta Data{Paper class{Evaluation Research}, Re...",https://doi.org/10.1109/ICSA51549.2021.00017,Architectural decay imposes real costs in term...,"[0.015215540304780006, -0.02269367128610611, 0..."
...,...,...,...,...,...,...,...
148,"Keim, Jan and Schulz, Sophie and Fuch{\ss}, Do...",Trace {Link} {Recovery} for {Software} {Archit...,10.1007/978-3-030-86044-8_7,"Meta Data{Research Level{Primary Research}, Ki...",NaN,Software Architecture Documentation often cons...,"[0.00465840520337224, 0.004803537856787443, 0...."
149,"Shabelnyk, Oleksandr and Frangoudis, Pantelis ...",Updating {Service}-{Based} {Software} {Systems...,10.1007/978-3-030-86044-8_10,"Meta Data{Paper class{Proposal of Solution}, R...",NaN,Contemporary component-based systems often man...,"[-0.004098787903785706, -0.006373200099915266,..."
150,Stefan Kugele and David Hettler and Jan Peter,Data-Centric Communication and Containerizatio...,10.1109/ICSA.2018.00016,"Meta Data{Kind{full}, Research Level{Primary R...",https://doi.org/10.1109/ICSA.2018.00016,Context: The functional interconnection and da...,"[-0.011806309223175049, 0.010385949164628983, ..."
151,Banani Roy and Amit Kumar Mondal and Chanchal ...,Towards a Reference Architecture for Cloud-Bas...,10.1109/ICSA.2017.42,"Meta Data{Paper class{Validation Research, Eva...",https://doi.org/10.1109/ICSA.2017.42,The domain of plant genotyping and phenotyping...,"[-0.00909524504095316, -0.012267849408090115, ..."


In [5]:
embeddings = np.array(df["embedding"].tolist())
print("Embeddings shape:", embeddings.shape)

Embeddings shape: (153, 1536)


### Install and start postgres

In [6]:
# !docker pull ankane/pgvector
!podman pull docker.io/ankane/pgvector

WARN[0000] The cgroupv2 manager is set to systemd but there is no systemd user session available 
WARN[0000] For using systemd, you may need to login using an user session 
WARN[0000] Alternatively, you can enable lingering with: `loginctl enable-linger 1030` (possibly as root) 
WARN[0000] Falling back to --cgroup-manager=cgroupfs    
WARN[0000] The cgroupv2 manager is set to systemd but there is no systemd user session available 
WARN[0000] For using systemd, you may need to login using an user session 
WARN[0000] Alternatively, you can enable lingering with: `loginctl enable-linger 1030` (possibly as root) 
WARN[0000] Falling back to --cgroup-manager=cgroupfs    
Error: short-name "ankane/pgvector" did not resolve to an alias and no unqualified-search registries are defined in "/etc/containers/registries.conf"
WARN[0000] Failed to add pause process to systemd sandbox cgroup: Process org.freedesktop.systemd1 exited with status 1 


In [7]:
# !docker run --rm --name postgres-vector -e POSTGRES_PASSWORD=hasenfurz -p "5432:5432" -d ankane/pgvector
!podman run --rm --name postgres-vector -e POSTGRES_PASSWORD=a640dcd1f666 -p "5432:5432" -d ankane/pgvector

WARN[0000] The cgroupv2 manager is set to systemd but there is no systemd user session available 
WARN[0000] For using systemd, you may need to login using an user session 
WARN[0000] Alternatively, you can enable lingering with: `loginctl enable-linger 1030` (possibly as root) 
WARN[0000] Falling back to --cgroup-manager=cgroupfs    
WARN[0000] The cgroupv2 manager is set to systemd but there is no systemd user session available 
WARN[0000] For using systemd, you may need to login using an user session 
WARN[0000] Alternatively, you can enable lingering with: `loginctl enable-linger 1030` (possibly as root) 
WARN[0000] Falling back to --cgroup-manager=cgroupfs    
Error: short-name "ankane/pgvector" did not resolve to an alias and no unqualified-search registries are defined in "/etc/containers/registries.conf"


In [10]:
# Sleep for a couple of seconds for the docker container to start properly
import time

time.sleep(2)

In [11]:
import psycopg2

# Establish a connection
connection = psycopg2.connect(
    user="postgres",
    password="a640dcd1f666",
    host="localhost",
    port=5432
)

cursor = connection.cursor()

In [12]:
# Check connection
cursor.execute("SELECT now()")
res = cursor.fetchall()
print(res)

[(datetime.datetime(2023, 8, 5, 15, 27, 30, 146793, tzinfo=datetime.timezone.utc),)]


In [13]:
cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")

In [14]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS papers (doi VARCHAR PRIMARY KEY, title VARCHAR, embedding vector(%s));
""", (embeddings.shape[1],))

In [15]:
for inx, row in df.iterrows():
    doi = row["doi"]
    title = row["title"]
    embedding = row["embedding"].tolist()

    cursor.execute("""
    insert into papers (doi, title, embedding)
    values (%s, %s, %s)
    returning doi;
    """, (doi, title, embedding))

    res = cursor.fetchall()
    print("Inserted:", res[0][0])

connection.commit()

Inserted: 10.1109/ICSA51549.2021.00020
Inserted: 10.1109/ICSA51549.2021.00010
Inserted: 10.1109/ICSA51549.2021.00022
Inserted: 10.1109/ICSA51549.2021.00014
Inserted: 10.1109/ICSA51549.2021.00017
Inserted: 10.1109/ICSA51549.2021.00009
Inserted: 10.1109/ICSA51549.2021.00012
Inserted: 10.1109/ICSA51549.2021.00018
Inserted: 10.1109/ICSA51549.2021.00021
Inserted: 10.1109/ICSA51549.2021.00024
Inserted: 10.1109/ICSA51549.2021.00016
Inserted: 10.1109/ICSA51549.2021.00023
Inserted: 10.1109/ICSA51549.2021.00013
Inserted: 10.1109/ICSA47634.2020.00022
Inserted: 10.1109/ICSA47634.2020.00018
Inserted: 10.1109/ICSA47634.2020.00016
Inserted: 10.1109/ICSA47634.2020.00013
Inserted: 10.1109/ICSA47634.2020.00010
Inserted: 10.1109/ICSA47634.2020.00009
Inserted: 10.1109/ICSA47634.2020.00012
Inserted: 10.1109/ICSA47634.2020.00026
Inserted: 10.1109/ICSA47634.2020.00011
Inserted: 10.1109/ICSA47634.2020.00019
Inserted: 10.1109/ICSA47634.2020.00014
Inserted: 10.1109/ICSA47634.2020.00024
Inserted: 10.1109/ICSA476

In [16]:
connection.commit()

In [17]:
example = df.iloc[0]
print("Title:", example["title"])
print("doi:", example["doi"])
print()

embedding = example["embedding"].tolist()

with connection.cursor() as cur:
    cur.execute("SELECT doi, title FROM papers ORDER BY embedding <-> %s::vector LIMIT 5;", (embedding,))
    papers = cur.fetchall()
    print("Similar papers:")
    
    for paper in papers:
        print(f"- {paper[0]}: {paper[1]}")

Title: Aligning Architecture with Business Goals in the Automotive Domain
doi: 10.1109/ICSA51549.2021.00020
Similar papers:
- 10.1109/ICSA51549.2021.00020: Aligning Architecture with Business Goals in the Automotive Domain
- 10.1109/ICSA47634.2020.00013: Technical Architectures for Automotive Systems
- 10.1109/ICSA.2017.20: On Service-Orientation for Automotive Software
- 10.1109/ICSA.2019.00025: On Interfaces to Support Agile Architecting in Automotive: An Exploratory Case Study
- 10.1007/978-3-030-58923-3\_2: Formal Software Architectural Migration Towards Emerging Architectural Styles
